# 匯入可以調用R語言的套件

In [1]:
'''
程式碼參考網址：https://stackoverflow.com/questions/63863449/oserror-cannot-load-library-c-program-files-r-r-4-0-2-bin-x64-r-dll-error-0
'''
import os
os.environ["R_HOME"] = r"C:\Program Files\R\R-4.3.1"
os.environ["PATH"]   = r"C:\Program Files\R\R-4.3.1\bin\x64" + ";" + os.environ["PATH"]
import rpy2
from rpy2.robjects import pandas2ri, packages
pandas2ri.activate()
stats = packages.importr('stats')

# 此為調用R的重要套件
import rpy2.robjects as robjects

# 前置作業

In [2]:
"""
要去找套件(conversion)改編碼的程式碼
路徑 -> C:\\Users\\M11021001\\Anaconda3\\Lib\\site-packages\\rpy2\\rinterface_lib

參考網址裡的編碼是GBK，這邊要改成BIG5才能使用
參考網址：https://blog.csdn.net/qq_44645101/article/details/127069531

要先下載JAVA才能用rJava、RWeka
設定路徑參考網址：https://www.796t.com/content/1548774361.html
import套件前要去RStudio先下載套件
"""

'\n要去找套件(conversion)改編碼的程式碼\n路徑 -> C:\\Users\\M11021001\\Anaconda3\\Lib\\site-packages\\rpy2\\rinterface_lib\n\n參考網址裡的編碼是GBK，這邊要改成BIG5才能使用\n參考網址：https://blog.csdn.net/qq_44645101/article/details/127069531\n\n要先下載JAVA才能用rJava、RWeka\n設定路徑參考網址：https://www.796t.com/content/1548774361.html\nimport套件前要去RStudio先下載套件\n'

# 載入套件及資料集(已處理的)

In [4]:
# 參考網址：https://www.twblogs.net/a/5d8cade9bd9eee541c34c273
# 參考網址：https://rstudio-pubs-static.s3.amazonaws.com/275285_90aaf9a2a64d43a5846a86dbcde8eba9.html

# 這邊是R所需要的套件包
rPackage = '''
library(rattle)
library(rpart)
library(rpart.plot)
library(rJava)
library(RWeka)  # 用這個套件前要先載rJava
library(party)
library(partykit)
library(C50)
library(export)
'''

# 載入經過前處理後的資料集
df = '''
dataname = "Adult" # 後面會用到
Data <- read.csv("C:/Users/M11021001/OneDrive - 國立雲林科技大學/Desktop/資料探勘/作業1/adult_clean_data.csv")
'''
robjects.r(rPackage)
robjects.r(df)

In [5]:
# 將資料集切成訓練集及測試集
set_data = '''
idcol.y <- ncol(Data)

set.seed(1010) #隨機抽樣設置種子
(id.train<-sample(nrow(Data),0.7*nrow(Data))) #抽樣函數
ds.train <- Data[ id.train,] #生成訓練集
ds.test  <- Data[-id.train,] #生成測試集
'''
robjects.r(set_data)

# CART

In [6]:
rCART = '''
model.cart<-rpart(income~., 
                  data=ds.train, 
                  method="class", 
                  parms=list( split="gini" ) ) # 使用CART算法的時候, split = “gini”
printcp( model.cart )

# 繪製決策樹圖形
# rpart.plot( model.cart, branch=1, type=2, fallen.leaves=F, cex=1, sub="決策樹模型-CART")
# 這邊參考網站使用fancyRpartPlot

png(file = paste("CART of", dataname, ".png"))
fancyRpartPlot( model.cart, 
                main = paste("CART of", dataname, "\n(", Sys.time(),")", sep = " "), )
dev.off()

#利用預測集進行預測
pred.cart <- predict( model.cart, 
                      newdata=ds.test, 
                      type="class" )   
#輸出混淆矩陣
table( ds.test$income, pred.cart, dnn=c("真實值","預測值") )    #輸出混淆矩陣
'''

robjects.r(rCART)


Classification tree:
rpart(formula = income ~ ., data = ds.train, method = "class", 
    parms = list(split = "gini"))

Variables actually used in tree construction:
[1] capital.gain  capital.loss  education.num relationship 

Root node error: 5237/21113 = 0.24805

n= 21113 

        CP nsplit rel error  xerror     xstd
1 0.114760      0   1.00000 1.00000 0.011983
2 0.060340      2   0.77048 0.77048 0.010909
3 0.047737      3   0.71014 0.71014 0.010570
4 0.020813      4   0.66240 0.66240 0.010281
5 0.013175      5   0.64159 0.64713 0.010185
6 0.010000      7   0.61524 0.62230 0.010024


6390,1041,388,1230


# ID3

In [7]:
rID3 = '''
model.ID3<-rpart(income~., 
                  data=ds.train, 
                  method="class", 
                  parms=list( split="information" ) ) # 使用ID3算法時候, split = “information”
printcp( model.ID3 )

# 繪製決策樹圖形
# rpart.plot( model.ID3, branch=1, type=2, fallen.leaves=F, cex=1, sub="決策樹模型-CART")
# 這邊參考網站使用fancyRpartPlot

png(file = paste("ID3 of", dataname, ".png"))
fancyRpartPlot( model.ID3, 
                main = paste("ID3 of", dataname, "\n(", Sys.time(),")", sep = " "), )
dev.off()

#利用預測集進行預測
pred.ID3 <- predict( model.ID3, 
                      newdata=ds.test, 
                      type="class" )   
#輸出混淆矩陣
table( ds.test$income, pred.ID3, dnn=c("真實值","預測值") )    #輸出混淆矩陣
'''

robjects.r(rID3)


Classification tree:
rpart(formula = income ~ ., data = ds.train, method = "class", 
    parms = list(split = "information"))

Variables actually used in tree construction:
[1] capital.gain  capital.loss  education.num relationship 

Root node error: 5237/21113 = 0.24805

n= 21113 

        CP nsplit rel error  xerror     xstd
1 0.114760      0   1.00000 1.00000 0.011983
2 0.060340      2   0.77048 0.76265 0.010866
3 0.023168      3   0.71014 0.69276 0.010467
4 0.010598      6   0.64063 0.65457 0.010232
5 0.010000      8   0.61944 0.63739 0.010123


6396,1056,382,1215


# C4.5

In [10]:
C45 = '''

# C4.5要把資料集轉換成Factor才可以算
ds.trainFac <- lapply(ds.train, as.factor)
ds.testFac <- lapply(ds.test, as.factor)

model.C45 <- J48(income~., 
                data=ds.trainFac)
                  
print(model.C45)

# 繪製決策樹圖形
# rpart.plot( model.C45, branch=1, type=2, fallen.leaves=F, cex=1, sub="決策樹模型-CART")
# 這邊參考網站使用fancyRpartPlot

png(file = paste("C4.5 of", dataname, ".png"))
plot( model.C45, 
      main =paste("C4.5 of", dataname, ".png"))
dev.off()

#利用預測集進行預測
pred.C45 <- predict( model.C45, 
                      newdata=ds.testFac, 
                      type="class" )   
#輸出混淆矩陣
table( ds.testFac$income, pred.C45, dnn=c("真實值","預測值") )    #輸出混淆矩陣
'''
robjects.r(C45)

J48 pruned tree
------------------

capital.gain = 0
|   marital.status = 0: 0 (2737.0/213.0)
|   marital.status = 0.166666667
|   |   relationship = 0: 0 (5.0/1.0)
|   |   relationship = 0.2: 0 (0.0)
|   |   relationship = 0.4: 0 (1.0)
|   |   relationship = 0.6: 0 (1.0)
|   |   relationship = 0.8: 0 (0.0)
|   |   relationship = 1: 1 (5.0/1.0)
|   marital.status = 0.333333333
|   |   capital.loss = 0
|   |   |   education = 0: 0 (205.0/26.0)
|   |   |   education = 0.066666667: 0 (215.0/17.0)
|   |   |   education = 0.133333333: 0 (78.0/14.0)
|   |   |   education = 0.2: 0 (44.0/4.0)
|   |   |   education = 0.266666667: 0 (87.0/2.0)
|   |   |   education = 0.333333333: 0 (203.0/17.0)
|   |   |   education = 0.4: 0 (142.0/12.0)
|   |   |   education = 0.466666667: 0 (244.0/111.0)
|   |   |   education = 0.533333333: 0 (377.0/146.0)
|   |   |   education = 0.6: 1 (1396.0/536.0)
|   |   |   education = 0.666666667: 1 (127.0/29.0)
|   |   |   education = 0.733333333: 0 (2794.0/773.0)
|   

6423,1324,355,947


# C5.0

In [12]:
C50 = '''

# C50要把class轉換成Factor才可以算
df <- Data
df$income <- as.factor(df$income)

idcol.y <- ncol(df)
set.seed(1010) #隨機抽樣設置種子
(id.train<-sample(nrow(df),0.7*nrow(df))) #抽樣函數
ds.trainC50 <- df[ id.train,] #生成訓練集
ds.testC50  <- df[-id.train,] #生成測試集

model.C50 <- C5.0(income ~.,
                               data=ds.trainC50)
                  
#規則輸出
summary( model.C50 )

png(file = paste("C5.0 of", dataname, ".png"))
plot( model.C50, 
      main =paste("C5.0 of", dataname, ".png"))
dev.off()

#利用預測集進行預測
pred.C50 <- predict( model.C50, 
                      newdata=ds.testC50, 
                      type="class" )   
#輸出混淆矩陣
table( ds.testC50$income, pred.C50, dnn=c("真實值","預測值") )
'''

robjects.r(C50)

6394,884,384,1387


In [13]:
# 模型準確率

acc_CART = '''
confus.matrix <- table(real=ds.test$income, predict=pred.cart)
sum(diag(confus.matrix))/sum(confus.matrix) # 對角線的數量/總數量
'''

acc_ID3 = '''
confus.matrix <- table(real=ds.test$income, predict=pred.ID3)
sum(diag(confus.matrix))/sum(confus.matrix) # 對角線的數量/總數量
'''

acc_C45 = '''
confus.matrix <- table(real=ds.testFac$income, predict=pred.C45)
sum(diag(confus.matrix))/sum(confus.matrix) # 對角線的數量/總數量
'''

acc_C50 = '''
confus.matrix <- table(real=ds.testC50$income, predict=pred.C50)
sum(diag(confus.matrix))/sum(confus.matrix) # 對角線的數量/總數量
'''

print("CART's Accurancy:", robjects.r(acc_CART))
print("ID3's Accurancy:", robjects.r(acc_ID3))
print("C45's Accurancy:", robjects.r(acc_C45))
print("C50's Accurancy:", robjects.r(acc_C50))

CART's Accurancy: [0.842082]
ID3's Accurancy: [0.84108741]
C45's Accurancy: [0.81445464]
C50's Accurancy: [0.85987402]
